## Big Data Day 2 Afternoon Assignment

In this assignment, we will learn about machine learning with Dask. We will use the market basket dataset loaded below and cluster our data.

In [1]:
!pip install dask[dataframe] --upgrade --quiet
!pip install dask-ml --quiet
!pip install aiohttp --quiet
!pip install joblib --quiet
!pip install dask distributed --upgrade --quiet
!pip install -U ipykernel --quiet
!pip install scikit-learn==0.23.2 --quiet

In [2]:
import dask.dataframe as dd
import numpy as np
import pandas as pd

In [3]:
market_basket = dd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Big%20Data/Mall_Customers.csv')

In [4]:
market_basket.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


To cluster our data, we must first examine and process it. The first step is to check for missing data. Do this in the cell below. If there is missing data, drop all rows containing missing data.

In [5]:
# Answer below:
market_basket.isnull().sum(0).compute()


CustomerID                0
Gender                    0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64

Next, we will get rid of the customer ID column. Drop the column and assign the resulting dataframe to a new variable.

In [6]:
# Answer below:
mb = market_basket.drop('CustomerID',axis=1)
mb.head()


,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,Male,19,15,39
1,Male,21,15,81
2,Female,20,16,6
3,Female,23,16,77
4,Female,31,17,40


We will now create a dummy variable from the gender variable.

In [7]:
# Answer below:
from dask_ml.preprocessing import OneHotEncoder, DummyEncoder
mb2 = dd.get_dummies(data=mb.categorize(),columns=['Gender'],drop_first=True)
mb2.head()

,Age,Annual Income (k$),Spending Score (1-100),Gender_Female
0,19,15,39,0
1,21,15,81,0
2,20,16,6,1
3,23,16,77,1
4,31,17,40,1


Since the data has different scales, we will scale all columns using min max scaling. We will write our own min max scaling function since the minmax scaler will produce numpy arrays instead of dask arrays. Recall that min max scaling requires finding the min and the max. We subtract the min from each observation and divide by the difference between the max and the min. Complete the function below.

In [8]:
# Answer below:

def dask_min_max(x):
    #Input: dask column
    min_array = x.min(axis=0)
    max_array = x.max(axis=0)
    #Output: dask column that has been min max scaled
    return (x - min_array) / (max_array - min_array)


Transform all columns using the scaler you wrote above.

In [9]:
# Answer below:
mb_scaled = dask_min_max(mb2)


In [10]:
mb_scaled['Annual Income (k$)'].min().compute(), mb_scaled['Annual Income (k$)'].max().compute()

(0.0, 1.0)

In [11]:
mb_scaled['Age'].min().compute(), mb_scaled['Age'].max().compute()

(0.0, 1.0)

Import the dask clustering function and cluster the data. Use 3 clusters and print the centroids.

In [12]:
# Answer below
from dask_ml.cluster import KMeans

kmeans = KMeans(n_clusters=3)

kmeans.fit(mb_scaled)

centers = pd.DataFrame(kmeans.cluster_centers_,columns=mb_scaled.columns)

kmeans_3_cluster = centers.style.background_gradient()
kmeans_3_cluster


,Age,Annual Income (k$),Spending Score (1-100),Gender_Female
0,0.668040,0.387002,0.302478,0.000000
1,0.386504,0.362705,0.515579,1.000000
2,0.192308,0.387206,0.651287,0.000000
